# Pipeline flow
1.load the dataset
2.Data preprocessing
3.Training Process
    a.create the model
    b.forward pass
    c.loss calculation 
    d. backprop
    e.parameters update
4.model evaluation 

In [90]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn

df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [91]:
df.drop(columns = ['id','Unnamed: 32'],inplace =True)

In [92]:
#train test

In [93]:
X_train , X_test , y_train , y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size = 0.2)

In [94]:
scaler = StandardScaler()

In [95]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [96]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [97]:
#numpy aray to pytorch

In [98]:
X_train.shape

(455, 30)

In [99]:

X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor= torch.FloatTensor(y_train)
y_test_tensor= torch.FloatTensor(y_test)

In [100]:
X_test_tensor.dtype

torch.float32

In [101]:
#let's define the Structure of the neural network 

In [102]:
class MySimpleNN():
    def __init__(self,X):
        self.weights = torch.rand(X.shape[1],1,dtype = torch.float32, requires_grad = True)
        self.bias = torch.zeros(1,dtype = torch.float32,requires_grad = True)

    def forward_pass(self,X):
        z = torch.matmul(X,self.weights) +self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss_function(self,y_pred,y):

        epsilon =  1e-7
        y_pred = torch.clamp(y_pred,epsilon, 1-epsilon)

        loss = -(y_train_tensor * torch.log(y_pred) + (1-y_train_tensor) * torch.log(1-y_pred)).mean()
        return loss

In [103]:
l_r = 0.1
epochs = 25


In [104]:
#training_pipeline
model = MySimpleNN(X_train_tensor)

for epochs in range(epochs):

    #forward_pass 
    y_pred = model.forward_pass(X_train_tensor)

    #calculate loss 

    loss = model.loss_function(y_pred,y_train_tensor)
    print(f'epoch :{epochs+1} ,Loss = {loss.item()}')

    #backward pass 
    loss.backward()
    # calculates all the derivates 

    #parameters update 
    with torch.no_grad(): # dont track the gradients 
        
        
        model.weights -= l_r + model.weights.grad
        model.bias  -= l_r + model.bias.grad
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    
    


epoch :1 ,Loss = 3.7045276165008545
epoch :2 ,Loss = 1.5610004663467407
epoch :3 ,Loss = 1.364208698272705
epoch :4 ,Loss = 0.7257760763168335
epoch :5 ,Loss = 0.8289128541946411
epoch :6 ,Loss = 0.7319345474243164
epoch :7 ,Loss = 0.8289925456047058
epoch :8 ,Loss = 0.7629913091659546
epoch :9 ,Loss = 0.8239476680755615
epoch :10 ,Loss = 0.7948721647262573
epoch :11 ,Loss = 0.8262375593185425
epoch :12 ,Loss = 0.8194279074668884
epoch :13 ,Loss = 0.8349423408508301
epoch :14 ,Loss = 0.8371556401252747
epoch :15 ,Loss = 0.8460652232170105
epoch :16 ,Loss = 0.8509200811386108
epoch :17 ,Loss = 0.8573818206787109
epoch :18 ,Loss = 0.8626477122306824
epoch :19 ,Loss = 0.8681449294090271
epoch :20 ,Loss = 0.8732345104217529
epoch :21 ,Loss = 0.8782532215118408
epoch :22 ,Loss = 0.8830790519714355
epoch :23 ,Loss = 0.8877942562103271
epoch :24 ,Loss = 0.8923860192298889
epoch :25 ,Loss = 0.8968799710273743


In [105]:
print(model.weights)
model.bias

tensor([[-6.6606e-01],
        [-2.6129e-01],
        [-2.9218e-01],
        [-6.7946e-01],
        [ 6.7016e-02],
        [ 2.2291e-01],
        [ 5.5125e-01],
        [ 1.6697e-01],
        [ 1.0066e-01],
        [-5.4003e-01],
        [-1.6782e-01],
        [-4.1338e-01],
        [ 3.3537e-01],
        [-2.7113e-01],
        [-2.7493e-01],
        [-2.9325e-04],
        [-2.7625e-01],
        [-7.7209e-02],
        [-4.2299e-01],
        [-9.2377e-02],
        [ 1.4471e-01],
        [ 5.4814e-02],
        [-1.0732e-01],
        [-8.6288e-02],
        [-3.3270e-01],
        [ 2.1299e-01],
        [ 7.1319e-02],
        [ 2.7909e-01],
        [-4.6272e-01],
        [-5.7062e-01]], requires_grad=True)


tensor([-1.2800], requires_grad=True)

In [106]:
#EVAluation

In [107]:
with torch.no_grad():
    y_pred = model.forward_pass(X_test_tensor)

    y_pred = (y_pred > 0.7).float()

    accuracy = (y_pred == y_test_tensor).float().mean()
print(f'Accuracy :{accuracy.item()}')




Accuracy :0.669282853603363


In [108]:
#now creating this using pytorch NN module

In [115]:
#inherit from the parent class 

class Model(nn.Module):

    def __init__(self,num_features):
        super().__init__()
        self.fc1 = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,features):
        out = self.fc1(features)
        out = self.sigmoid(out)
        
        return out 
    
    

In [116]:
features = torch.rand(10,5)

In [117]:
model = Model(features.shape[1])

In [119]:
model.forward(features)

tensor([[0.7513],
        [0.7497],
        [0.7110],
        [0.6984],
        [0.6836],
        [0.6519],
        [0.7157],
        [0.6553],
        [0.7589],
        [0.6879]], grad_fn=<SigmoidBackward0>)

In [122]:
model.fc1.weight

Parameter containing:
tensor([[ 0.3032,  0.0862, -0.0414,  0.4354,  0.3606]], requires_grad=True)

In [130]:
class model_2(nn.Module):
    def __init__(self,X):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.network = nn.Sequential(
            nn.Linear(X.shape[1],1),
            nn.Sigmoid(),
            
        )
    def forward(self,X):
        #x = self.flatten(X)
        logits = self.network(X)
        return logits

mo = model_2()

In [131]:
mo = model_2(X_train_tensor)

In [136]:
mo.forward(X_train_tensor)

tensor([[0.4057],
        [0.4489],
        [0.4885],
        [0.3209],
        [0.8495],
        [0.3173],
        [0.3127],
        [0.3592],
        [0.2745],
        [0.9476],
        [0.4287],
        [0.3477],
        [0.3101],
        [0.2705],
        [0.5278],
        [0.2736],
        [0.4960],
        [0.7700],
        [0.3713],
        [0.3564],
        [0.3151],
        [0.3073],
        [0.8716],
        [0.4463],
        [0.7459],
        [0.6756],
        [0.4177],
        [0.4145],
        [0.3813],
        [0.6445],
        [0.1717],
        [0.4091],
        [0.3116],
        [0.3567],
        [0.5464],
        [0.3527],
        [0.3415],
        [0.6149],
        [0.2167],
        [0.3517],
        [0.3273],
        [0.3430],
        [0.6568],
        [0.2993],
        [0.3552],
        [0.3116],
        [0.5189],
        [0.4599],
        [0.9562],
        [0.2321],
        [0.5905],
        [0.2228],
        [0.5932],
        [0.2787],
        [0.3892],
        [0

In [ ]:
what